<a href="https://colab.research.google.com/github/T-mang/hello/blob/master/spark_hive_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install openjdk-8-jre-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

import os
from os.path import join, abspath
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

!ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

from pyspark.sql import Row
warehouse_location = abspath('spark-warehouse')

spark = SparkSession \
      .builder \
      .config("spark.sql.warehouse.dir", warehouse_location) \
      .enableHiveSupport() \
      .getOrCreate()
spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:15 http

In [3]:
original_df = spark.read.csv("original.csv", inferSchema = True, header = True)
original_df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [4]:
original_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- City: string (nullable = true)
 |-- JobTitle: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)



In [5]:
original_df.columns

['id',
 'first_name',
 'last_name',
 'gender',
 'City',
 'JobTitle',
 'Salary',
 'Latitude',
 'Longitude']

In [6]:
len(original_df.columns)

9

In [7]:
original_df.count()

1000

In [8]:
original_df.createTempView('original_src')

In [9]:
spark.sql("select * from original_src").show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

eliminate rows with nulls in jobtitle and city columns
replace nulls in latitude/longitude with median value
convert salary to decimal

In [13]:
df1 = spark.sql("""select  id, \
    first_name, \
    last_name, \
    gender, \
    city, \
    JobTitle as jobtitle, \
    cast(substr(Salary,2) as decimal(9,2)) as salary, \
    Latitude  as latitude, \
    Longitude as longitude
    from original_src \
    where LENGTH(LTRIM(jobtitle)) <> 0  \
      and LENGTH(LTRIM(city)) <> 0
    """)
df1.show()

+---+----------+----------+------+---------------+--------------------+--------+----------+-----------+
| id|first_name| last_name|gender|           city|            jobtitle|  salary|  latitude|  longitude|
+---+----------+----------+------+---------------+--------------------+--------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|62846.60|48.8231572|103.5218199|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|30101.16|53.4266145| -6.1644997|
|  8|   Goddart|     Flear|  Male|      Trélissac|Desktop Support T...|46116.36|45.1905186|  0.7423124|
|  9|      Roth|O'Cannavan|  Male|         Heitan|VP Product Man

In [11]:
spark.sql("""select count(case when isnull(latitude) then 1  else null \
                                end ) latitude_nulls, \
                    count(case when isnull(longitude) then 1  else null \
                                end ) longitude_nulls            
                                from original_src """).show()

+--------------+---------------+
|latitude_nulls|longitude_nulls|
+--------------+---------------+
|             1|              0|
+--------------+---------------+



In [15]:
import numpy as np

latitudes = df1.select('latitude').filter(df1.latitude.isNotNull())
latitudes.show(5)

+----------+
|  latitude|
+----------+
|50.5774075|
|48.8231572|
|44.5047212|
|53.4266145|
|45.1905186|
+----------+
only showing top 5 rows



In [16]:
median = np.median(latitudes.collect())
print(median)

31.9339724


In [17]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import *
df_fin = df1.withColumn('latitude1', when(df1.latitude.isNull(), lit(median)).otherwise(df1.latitude)).drop(df1.latitude)

df_fin.show()
df_fin.createOrReplaceTempView("table1")

+---+----------+----------+------+---------------+--------------------+--------+-----------+----------+
| id|first_name| last_name|gender|           city|            jobtitle|  salary|  longitude| latitude1|
+---+----------+----------+------+---------------+--------------------+--------+-----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|57438.18| 16.4967184|50.5774075|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|62846.60|103.5218199|48.8231572|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|61489.23| 38.1300171|44.5047212|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|63863.09| 37.6489954|31.9339724|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|30101.16| -6.1644997|53.4266145|
|  8|   Goddart|     Flear|  Male|      Trélissac|Desktop Support T...|46116.36|  0.7423124|45.1905186|
|  9|      Roth|O'Cannavan|  Male|         Heitan|VP Product Man

In [18]:
fin_tab = spark.sql(""" select id,first_name,last_name,gender,city,jobtitle,salary,latitude1 as latitude,longitude \
from table1""")
fin_tab.show()

+---+----------+----------+------+---------------+--------------------+--------+----------+-----------+
| id|first_name| last_name|gender|           city|            jobtitle|  salary|  latitude|  longitude|
+---+----------+----------+------+---------------+--------------------+--------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|62846.60|48.8231572|103.5218199|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|63863.09|31.9339724| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|30101.16|53.4266145| -6.1644997|
|  8|   Goddart|     Flear|  Male|      Trélissac|Desktop Support T...|46116.36|45.1905186|  0.7423124|
|  9|      Roth|O'Cannavan|  Male|         Heitan|VP Product Man

In [19]:
fin_tab.printSchema()

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- city: string (nullable = true)
 |-- jobtitle: string (nullable = true)
 |-- salary: decimal(9,2) (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [23]:
spark.sql("create schema if not exists test")
fin_tab.write.format('parquet').mode("overwrite").saveAsTable('test.clean_source')

In [25]:
spark.sql("""select gender,avg(salary) as avg_salary \
           from table1 group by gender""").show()

+------+------------+
|gender|  avg_salary|
+------+------------+
|Female|55677.250184|
|  Male|55361.093839|
+------+------------+



In [36]:
df_salary = spark.sql(""" select gender,jobtitle, avg(salary) as avg_salary \
             from table1 group by gender,jobtitle order by jobtitle, gender""")
df_salary.show()

+------+--------------------+------------+
|gender|            jobtitle|  avg_salary|
+------+--------------------+------------+
|Female| Account Coordinator|46707.445000|
|  Male| Account Coordinator|51446.623333|
|Female|   Account Executive|52020.780000|
|  Male|   Account Executive|65415.968000|
|Female|Account Represent...|51116.973333|
|Female|Account Represent...|41786.910000|
|  Male|Account Represent...|40562.700000|
|  Male|Account Represent...|22420.710000|
|Female|       Accountant II|50354.815000|
|Female|      Accountant III|15589.560000|
|  Male|      Accountant III|39183.796667|
|Female|       Accountant IV|82732.250000|
|Female|Accounting Assist...|58916.085000|
|  Male|Accounting Assist...|59255.430000|
|Female|Accounting Assist...|44071.866667|
|  Male|Accounting Assist...|18795.440000|
|Female|Accounting Assist...|57337.485000|
|Female|             Actuary|54196.860000|
|  Male|             Actuary|75540.217500|
|Female|Administrative As...|43652.280000|
+------+---

In [41]:
df_salary.createOrReplaceTempView("salaries")
spark.sql("select * from salaries limit 3").show()

+------+-------------------+------------+
|gender|           jobtitle|  avg_salary|
+------+-------------------+------------+
|Female|Account Coordinator|46707.445000|
|  Male|Account Coordinator|51446.623333|
|Female|  Account Executive|52020.780000|
+------+-------------------+------------+



In [48]:
spark.sql("""select f.jobtitle,m.jobtitle,f.avg_salary as f_sal,\
             m.avg_salary as m_sal, f.avg_salary - m.avg_salary as delta \
             from (select gender, jobtitle, avg_salary from salaries where gender = 'Female') f \
             join (select gender, jobtitle, avg_salary from salaries where gender = 'Male') m \
             on (f.jobtitle = m.jobtitle) \
             """).show()

+--------------------+--------------------+------------+------------+-------------+
|            jobtitle|            jobtitle|       f_sal|       m_sal|        delta|
+--------------------+--------------------+------------+------------+-------------+
| Account Coordinator| Account Coordinator|46707.445000|51446.623333| -4739.178333|
|   Account Executive|   Account Executive|52020.780000|65415.968000|-13395.188000|
|Account Represent...|Account Represent...|41786.910000|40562.700000|  1224.210000|
|      Accountant III|      Accountant III|15589.560000|39183.796667|-23594.236667|
|Accounting Assist...|Accounting Assist...|58916.085000|59255.430000|  -339.345000|
|Accounting Assist...|Accounting Assist...|44071.866667|18795.440000| 25276.426667|
|             Actuary|             Actuary|54196.860000|75540.217500|-21343.357500|
|Administrative As...|Administrative As...|43652.280000|65910.310000|-22258.030000|
|Administrative As...|Administrative As...|68744.575000|29440.470000| 39304.

In [50]:
spark.sql("select city, avg(salary) avg_salary from table1 group by city order by avg_salary").show()

+-------------------+------------+
|               city|  avg_salary|
+-------------------+------------+
|             Gaoshi|10101.920000|
|             Jarash|10210.850000|
|            Połomia|10616.440000|
|           Paris 13|10760.340000|
|         Santa Rita|10766.430000|
| San Pedro Masahuat|10808.160000|
|           Zhizhong|10913.500000|
|              Ursus|11014.990000|
|            Baoshui|11084.340000|
|            Tiannan|11209.810000|
|Venustiano Carranza|11223.750000|
|             Puyuan|11224.170000|
|     Mariano Moreno|11376.580000|
|           Cipinang|11617.510000|
|   Kham Sakae Saeng|11630.950000|
|       Bajingmeduro|11652.990000|
|         Concepción|11824.520000|
|            Lachute|11926.310000|
|           Tiandiba|11950.080000|
|             Podhum|12016.070000|
+-------------------+------------+
only showing top 20 rows

